In [8]:
import requests
def run_solver(domain_file, problem_file, solver = "dual-bfws-ffparser"):

    req_body = {"domain" : domain_file, "problem" : problem_file}

    # Send job request to solve endpoint
    solve_request_url=requests.post(f"https://solver.planning.domains:5001/package/{solver}/solve", json=req_body).json()
    print(solve_request_url)
    # Query the result in the job
    celery_result=requests.post('https://solver.planning.domains:5001' + solve_request_url['result'])
    print(celery_result.json())

    while celery_result.json().get("status","")== 'PENDING':
        # Query the result every 0.5 seconds while the job is executing
        celery_result=requests.post('https://solver.planning.domains:5001' + solve_request_url['result'])

    result = celery_result.json()['result']
    return result


In [1]:
# read domain.pddl
import os


with open("domain.pddl", "r") as f:
    domain_pddl = f.read()
with open("problem.pddl", "r") as f:
    problem_pddl = f.read()

In [9]:
run_solver(domain_pddl, problem_pddl)

{'result': '/check/10c0c439-b1ed-46c6-ba06-b478ea95df51?external=True'}
{'result': {'call': 'timeout 30 planutils run dual-bfws-ffparser -- domain problem plan', 'output': {'plan': ''}, 'output_type': 'generic', 'stderr': "\ndomain: syntax error in line 4, ':PRECONDITIONS':\ndomain definition expected\n", 'stdout': ''}, 'status': 'ok'}


{'call': 'timeout 30 planutils run dual-bfws-ffparser -- domain problem plan',
 'output': {'plan': ''},
 'output_type': 'generic',
 'stderr': "\ndomain: syntax error in line 4, ':PRECONDITIONS':\ndomain definition expected\n",
 'stdout': ''}

In [2]:
import re

def format_pddl(pddl_text):
    """Format PDDL text with proper indentation and line breaks"""
    # Add line breaks after key PDDL keywords
    formatted = pddl_text.replace('(define', '\n(define')
    formatted = formatted.replace('(:requirements', '\n  (:requirements')
    formatted = formatted.replace('(:types', '\n  (:types')
    formatted = formatted.replace('(:action', '\n  (:action')
    formatted = formatted.replace(':parameters', '\n    :parameters')
    formatted = formatted.replace(':preconditions', '\n    :preconditions')
    formatted = formatted.replace(':effects', '\n    :effects')
    
    # Add line breaks before closing major sections
    formatted = formatted.replace('))', ')\n)')
    
    # Clean up extra whitespace and ensure proper spacing
    lines = formatted.split('\n')
    cleaned_lines = []
    for line in lines:
        line = line.strip()
        if line:
            cleaned_lines.append(line)
    
    return '\n'.join(cleaned_lines)

# Format and print the domain_pddl
print("PDDL Domain (Formatted):")
print("=" * 50)
formatted_domain = format_pddl(domain_pddl)
print(formatted_domain)
print("=" * 50)

PDDL Domain (Formatted):
(define (domain exploration)
(:requirements :strips :typing)
(:types location direction)
(:action open-door
:parameters (?loc1 - location ?loc2 - location ?dir - direction)
:preconditions (and (at ?loc1) (door-closed ?loc1 ?loc2 ?dir)
)
:effects (door-open ?loc1 ?loc2 ?dir)
)
(:action move
:parameters (?from - location ?to - location ?dir - direction)
:preconditions (and (at ?from) (door-open ?from ?to ?dir)
)
:effects (at ?to)
))


In [142]:
import sys
# print(sys.path)
import time
from datetime import date
import csv
import json
import asyncio
import re

## module for server
# from kani import Kani
# from kani.engines.huggingface import HuggingEngine

import subprocess
import requests

import os
import json
import glob
import random
import argparse
from os.path import join as pjoin

from textworld_express import TextWorldExpressEnv
import textworld
import textworld.gym

from alfworld.info import ALFWORLD_DATA
from alfworld.agents.utils.misc import add_task_to_grammar
from alfworld.agents.environment.alfred_tw_env import AlfredExpert, AlfredDemangler, AlfredExpertType

from openai import OpenAI

In [89]:
env = TextWorldExpressEnv(envStepLimit=100)
NUM_LOCATIONS_lst = [3,5,7,9,11]
seed_lst = range(1,11)
NUM_LOCATIONS = NUM_LOCATIONS_lst[1]

In [94]:
env.load(gameName="coin", gameParams=f"numLocations={NUM_LOCATIONS},numDistractorItems=2, includeDoors=1, limitInventorySize=0")
obs, infos = env.reset(seed=seed_lst[0], gameFold="train", generateGoldPath=True)

In [141]:
env.getRunHistory()

{'properties': {'numDistractorItems': 2,
  'seed': 1,
  'numLocations': 5,
  'includeDoors': 1,
  'gameSet': 1,
  'limitInventorySize': 0},
 'finalScore': 0.0,
 'numSteps': 18,
 'history': [{'observation': 'You are in the kitchen. In one part of the room you see a stove. There is also an oven. You also see a fridge that is closed. In another part of the room you see a counter, that has nothing on it. In one part of the room you see a kitchen cupboard that is closed. There is also a cutlery drawer that is closed. You also see a trash can that is closed. In another part of the room you see a dishwasher that is closed. In one part of the room you see a dining chair, that has nothing on it. \nTo the South you see a closed plain door. To the West you see a closed sliding patio door. ',
   'look': 'You are in the kitchen. In one part of the room you see a stove. There is also an oven. You also see a fridge that is closed. In another part of the room you see a counter, that has nothing on it.

In [96]:
obs

'You are in the kitchen. In one part of the room you see a stove. There is also an oven. You also see a fridge that is closed. In another part of the room you see a counter, that has nothing on it. In one part of the room you see a kitchen cupboard that is closed. There is also a cutlery drawer that is closed. You also see a trash can that is closed. In another part of the room you see a dishwasher that is closed. In one part of the room you see a dining chair, that has nothing on it. \nTo the South you see a closed plain door. To the West you see a closed sliding patio door. '

In [97]:
print(infos['observation'])

You are in the kitchen. In one part of the room you see a stove. There is also an oven. You also see a fridge that is closed. In another part of the room you see a counter, that has nothing on it. In one part of the room you see a kitchen cupboard that is closed. There is also a cutlery drawer that is closed. You also see a trash can that is closed. In another part of the room you see a dishwasher that is closed. In one part of the room you see a dining chair, that has nothing on it. 
To the South you see a closed plain door. To the West you see a closed sliding patio door. 


In [98]:
valid_actions = sorted(infos['validActions'])
# valid_actions.remove('look around') # 이거 왜뺐지
# valid_actions.remove('inventory') # 이거 왜뺐지
valid_actions

['close door to south',
 'close door to west',
 'inventory',
 'look around',
 'move south',
 'move west',
 'open door to south',
 'open door to west']

In [99]:
env.getGoldActionSequence()

['look around',
 'open door to south',
 'open door to west',
 'move west',
 'take coin']

In [138]:
action = 'move north'
obs, reward, done, infos = env.step(action)

In [ ]:
obs # 결국 문 말고는 다른걸 여는 선택지는 없네(drawer, cupboard, trashcan, fridge, ...etc.)

'You are in the bedroom. In one part of the room you see a dressing table, that has nothing on it. There is also a desk chair, that has nothing on it. You also see a desk, that has nothing on it. In another part of the room you see a chest of drawers that is closed. In one part of the room you see a wardrobe that is closed. There is also a night stand, that has nothing on it. You also see a bed, that has nothing on it. \nThrough an open wood door, to the South you see the corridor. '

In [55]:
reward

0.0

In [45]:
done

True

In [140]:
infos

{'observation': 'You are in the bedroom. In one part of the room you see a dressing table, that has nothing on it. There is also a desk chair, that has nothing on it. You also see a desk, that has nothing on it. In another part of the room you see a chest of drawers that is closed. In one part of the room you see a wardrobe that is closed. There is also a night stand, that has nothing on it. You also see a bed, that has nothing on it. \nThrough an open wood door, to the South you see the corridor. ',
 'look': 'You are in the bedroom. In one part of the room you see a dressing table, that has nothing on it. There is also a desk chair, that has nothing on it. You also see a desk, that has nothing on it. In another part of the room you see a chest of drawers that is closed. In one part of the room you see a wardrobe that is closed. There is also a night stand, that has nothing on it. You also see a bed, that has nothing on it. \nThrough an open wood door, to the South you see the corridor

In [68]:
env.getRunHistory()['history']

[{'observation': 'You are in the kitchen. In one part of the room you see a stove. There is also an oven. You also see a fridge that is closed. In another part of the room you see a counter, that has nothing on it. In one part of the room you see a kitchen cupboard that is closed. There is also a cutlery drawer that is closed. You also see a trash can that is closed. In another part of the room you see a dishwasher that is closed. In one part of the room you see a dining chair, that has nothing on it. There is also a coin. \nTo the South you see the corridor. To the East you see a closed plain door. ',
  'look': 'You are in the kitchen. In one part of the room you see a stove. There is also an oven. You also see a fridge that is closed. In another part of the room you see a counter, that has nothing on it. In one part of the room you see a kitchen cupboard that is closed. There is also a cutlery drawer that is closed. You also see a trash can that is closed. In another part of the room

In [ ]:
valid_actions = sorted(infos['validActions'])
valid_actions.remove('look around') # 이거 왜뺐지
valid_actions.remove('inventory') # 이거 왜뺐지

print(f"Observations: {obs} \n") 
print(f"Gold path: {env.getGoldActionSequence()} \n")
print(f"Valid Actions: {valid_actions} \n")
print(f"taskDescription: {infos['taskDescription']} \n")

# actions = ['open door to south', 'move south', 'open door to west', 'move west', 'move east', 'move north', 'open door to west', 'move west', 'move east', 'move south', 'move south', 'move north', 'move east', 'open door to north', 'move north', 'take coin']
actions = ['open door to south']

for action in actions:
    print('>', action)
    obs, reward, done, infos = env.step(action)
    print(obs)

print(infos['done'])

In [149]:
max_retries=3
domain_file = open(f'domain.pddl').read()
problem_file = open(f'problem.pddl').read()

req_body = {"domain" : domain_file, "problem" : problem_file}

retries = 0

while retries < max_retries:
    try:
        # Send job request to solve endpoint
        solve_request_url = requests.post(
            f"https://solver.planning.domains:5001/package/dual-bfws-ffparser/solve", 
            json=req_body
        ).json()

        print(solve_request_url)

        # Query the result in the job
        celery_result = requests.post(
            'https://solver.planning.domains:5001' + solve_request_url['result']
        )
        print(celery_result.json().get("status", "?"))
        while celery_result.json().get("status", "") == 'PENDING':
            time.sleep(0.5)
            celery_result = requests.post(
                'https://solver.planning.domains:5001' + solve_request_url['result']
            )
        
        print(celery_result.json()['result'])
        break

    except Exception as e:
        print(f"Error encountered: {e}. Retrying in 5 seconds... (Attempt {retries+1}/{max_retries})")
        retries += 1
        time.sleep(5)

{'result': '/check/afb467d5-cd6a-4479-b03f-9b69792d742c?external=True'}
ok
{'call': 'timeout 30 planutils run dual-bfws-ffparser -- domain problem plan', 'output': {'plan': ''}, 'output_type': 'generic', 'stderr': "\ndomain: syntax error in line 1, ':PRECONDITIONS':\ndomain definition expected\n", 'stdout': ''}
ok
{'call': 'timeout 30 planutils run dual-bfws-ffparser -- domain problem plan', 'output': {'plan': ''}, 'output_type': 'generic', 'stderr': "\ndomain: syntax error in line 1, ':PRECONDITIONS':\ndomain definition expected\n", 'stdout': ''}


In [148]:
# domain_file
problem_file

'(define (problem patio-exploration) (:domain exploration) (:objects kitchen patio - location south west - direction) (:init (at kitchen) (door-closed kitchen patio south) (door-closed kitchen ?loc2 west)) (:goal (at patio)))'